## CT5135 Research Topics in AI

## Assignment 2

* Student ID(s): 22229358, 22230186, 20230220
* Student name(s): KOSTADIN GEORGIEV, YAMINI GIRKAR, SHUBHAM MANGLAM

### Imports

In [100]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers, activations

### Transform and Load Data

In [63]:
def walk_directory(dir, callback, label=None):
    for dirname, _, filenames in os.walk(dir):
        for filename in filenames:
            if filename.endswith(".jpg"):
                id = filename[:-4]
                pathname = os.path.join(dirname, filename)
                im = Image.open(pathname)
                imnp = np.array(im, dtype=float)

                if len(imnp.shape) != 3:
                    print("This is 1 channel, so we omit it",
                          imnp.shape, filename)
                    continue

                callback(id, imnp, label)

In [52]:
df = pd.DataFrame(columns=['height', 'width', 'channels', 'label'])

def get_img_shape(id, imnp, label):
    height, width, channel = imnp.shape
    df.loc[id] = [height, width, channel, label]

walk_directory('dataset/alpaca', get_img_shape, 'alpaca')
walk_directory('dataset/not_alpaca', get_img_shape, 'not_alpaca')

df

This is 1 channel, so we omit it (683, 1024) 4b9ef3ce2685ee32.jpg
This is 1 channel, so we omit it (914, 1024) 5a71db307230880e.jpg


,height,width,channels,label
01ad3ff1d94eb557,670,1024,3,alpaca
0346463867a297f4,768,1024,3,alpaca
038fae9e70c4c3f1,768,1024,3,alpaca
053608552d63f724,768,1024,3,alpaca
053dab62fbb47736,682,1024,3,alpaca
...,...,...,...,...
eb4d2e0c0252fdc2,682,1024,3,not_alpaca
eb4f5968c7866a3e,699,1024,3,not_alpaca
f4495f8511553631,680,1024,3,not_alpaca
f8d3c3d8be68c4fd,768,1024,3,not_alpaca


In [53]:
IMG_HEIGHT = df['height'].min()
IMG_WIDTH = df['width'].min()

print('Images will be resized to', IMG_HEIGHT, 'x', IMG_WIDTH)

Images will be resized to 481 x 583


In [54]:
def resize_images(label):
    source_dir = 'dataset/' + label
    target_dir = 'dataset_resized/'+ label
    subset = df.loc[df['label'] == label]

    for id, image in subset.iterrows():
        id = str(id)
        filename = id + ".jpg"
        
        source_path = os.path.join(source_dir, filename)
        target_path = os.path.join(target_dir, filename)
        
        image = Image.open(source_path)
        image = image.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
        image.save(target_path)

In [55]:
resize_images('alpaca')
resize_images('not_alpaca')

In [61]:
def load_images(dir):
    img_array = []
    
    walk_directory(dir, lambda id, imnp, label: img_array.append(imnp))

    return np.array(img_array)

In [64]:
img_alpaca = load_images('dataset_resized/alpaca')
img_not_alpaca = load_images('dataset_resized/not_alpaca')

In [70]:
print('alpaca:', img_alpaca.shape)
print('not_alpaca:', img_not_alpaca.shape)

alpaca: (142, 481, 583, 3)
not_alpaca: (183, 481, 583, 3)


### Implement Proposed Layer

In [121]:
#TODO: Finish and verfiy implementation
# Current implementation creates a regular Conv2D layer

class ProposedLayer(layers.Layer):
    def __init__(self, filters, receptive_field_size=(3, 3), activation=None, **kwargs):
        super(ProposedLayer, self).__init__(**kwargs)
        self.filters = filters
        self.receptive_field_size = receptive_field_size
        self.activation = activations.get(activation)
    
    def build(self, input_shape):
        n_samples, height, width, channels = input_shape

        self.kernel = self.add_weight(name='kernel',
                                      shape=(height, width, channels, self.filters),
                                      initializer='random_normal',
                                      trainable=True)
        self.bias = self.add_weight(name='bias',
                                    shape=(self.filters,),
                                    initializer='zeros',
                                    trainable=True)
        
        super(ProposedLayer, self).build(input_shape)

    def call(self, inputs):
        conv = tf.nn.conv2d(inputs, self.kernel, strides=(1, 1), padding='SAME')
        z = tf.nn.bias_add(conv, self.bias)
        a = self.activation(z)

        return a
    
    def compute_output_shape(self, input_shape):
        rf = self.receptive_field_size
        output_shape = list(input_shape)
        output_shape[1] = (input_shape[1] - rf[0]) // 1 + 1
        output_shape[2] = (input_shape[2] - rf[1]) // 1 + 1
        output_shape[3] = self.filters

        return tuple(output_shape)

### Create Model

In [123]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
n_classes = 2

# Input
inputs = layers.Input(input_shape)

# Block 1
x0 = ProposedLayer(16, receptive_field_size=(3, 3), activation='relu')(inputs)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x0)
# Block 2
x2 = ProposedLayer(12, receptive_field_size=(3, 3), activation='relu')(x1)
x3 = layers.MaxPooling2D(pool_size=(2, 2))(x2)
# Block 3
x4 = ProposedLayer(8, receptive_field_size=(3, 3), activation='relu')(x3)
x5 = layers.MaxPooling2D(pool_size=(2, 2))(x4)

# Flatten feature map - embedding size will become 34560
x6 = layers.Flatten()(x5)

# Dense layer for classification
# Start with units in dense layer = embedding_size / 64
x7 = layers.Dense(540, activation='relu')(x6)
# Output
outputs = layers.Dense(n_classes, activation='softmax')(x7)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 481, 583, 3)]     0         
                                                                 
 proposed_layer_33 (Proposed  (None, 481, 583, 16)     13460320  
 Layer)                                                          
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 240, 291, 16)     0         
 g2D)                                                            
                                                                 
 proposed_layer_34 (Proposed  (None, 240, 291, 12)     13409292  
 Layer)                                                          
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 120, 145, 12)     0         
 g2D)                                                      

### Compile

In [ ]:
#TODO

### Pre-Process Data

In [ ]:
#TODO

### Create Train/Test Splits

In [ ]:
#TODO

### Fit

In [ ]:
#TODO

### Evaluate

In [ ]:
#TODO

### Results and Discussion

In [ ]:
#TODO